# resp_evoked (quantifying response to evoked activity in a given session)
IMPORTANT: Keep this notebook as identical to `resp_photostim.ipynb` as possible.

Both scripts do 'trial-based' analysis, where each trial has some start/end time and identity.

The identity can be:
    1) evoked stimulation trial type (currently they are all the same)
    2) photostim stimulation trial type / the identity of the mark point being stimulated

The outputs for the 'resp_map-related' part are:
    1) 'response map'(s) of the movie field of view for each evoked trial (+summary over trials) - response of network to a particular trial type of evoked activity
    2) 'response map'(s) of the movie field of view for each photostim trial (+summary over trials) - response of network to a particular single cell stimulation

The outputs for the 'suite2p-related' part are:
    1) which cells are responding to a particular evoked stimulation
    2) which cells are responding to a particular photostim stimulation (the stimulated cell as well as all other cells)


In [ ]:
# channel: 2
# plane: 0
# frame_period: 0.033602476  # metadata-derived frame period for 30Hz acquisition
# fov_shape: [512, 512]      # shape of the FOV in pixels

# # baseline and response parameters
# bsln_n_frames: 10          # baseline window in frames
# resp_n_frames: 10          # response window in frames

# bsln_sub_type: "trial_by_trial"  # 'trial_by_trial' or 'session_wide'

# # spatial extent of response
# n_dist_bins: 724           # number of distance bins (724 = diagonal of 512x512)

# # visualization parameters
# n_rows_fov: 4
# vlim: 200
# txt_shift: [7, 7]
# sat_perc_fov: 99.99
# peristim_wind: [10, 30]
# zoomin_npix: 128

# dist_bins_xlim: 362        # 724 // 2
# dist_bins_xlim_zoom: 45    # 724 // 16

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from photostim_deve.response.io import get_all_tiff_paths, parse_evoked_protocol_csv
from photostim_deve.image_analysis.compute import get_resp_imgs
from photostim_deve.image_analysis.plot import plot_resp_imgs

%reload_ext autoreload
%autoreload 2

In [ ]:
# params (based on resp_photostim_config.yaml)
channel = 2
plane = 0 
frame_period = 0.033602476 # metadata-derived frame period for 30Hz acquisition fov_shape = [512, 512] # shape of the FOV in pixels
fov_shape = [512, 512] # shape of the FOV in pixels

# suite2p loading paerameters
n_planes = 1
fs = 1/frame_period
act_type = 'dff'

# skipping the rest of the parameters...

In [ ]:
bsln_dur = 500 # baseline duration in ms (in suite2p response this will also determine the number of frames after res_dur)
resp_dur = 2000 # response duration in ms

frame_avg_mode = 'median'
trial_avg_mode = 'median'

plot_debug = False # whether to plot the baseline and response time periods and images for each trial for debugging purposes

In [ ]:
# set params
subject = 'jm064'
session = '2025-11-18_s'

In [ ]:
session_path = os.path.join('data_proc', 'jm', subject, session)

# tiff file paths
s2p_path = os.path.join(session_path, 'suite2p', f'plane{plane}')
tiff_dir = os.path.join(s2p_path, f'reg_tif_chan{channel}')
all_tiff_paths = get_all_tiff_paths(tiff_dir)

# stimulation protocol pathsa
csv_save_path = os.path.join('data_proc', 'jm', subject, session, 'evoked_protocol.csv')
csv_load_path = csv_save_path

# output paths
output_path = os.path.join(session_path, 'resp_evoked')
output_fig_path = os.path.join(output_path, 'fig')

if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(output_fig_path):
    os.makedirs(output_fig_path)



In [ ]:
stim_times, stim_frames, stim_type = parse_evoked_protocol_csv(session_path, 
                                                               csv_load_path, 
                                                               frame_period=frame_period)

In [ ]:
# resp_bsln, resp_resp, resp_diff, f_mean = get_resp_imgs(all_tiff_paths, 
#                                                 stim_frames, 
#                                                 stim_type, 
#                                                 frame_avg_mode=frame_avg_mode, 
#                                                 bsln_dur=bsln_dur, 
#                                                 resp_dur=resp_dur, 
#                                                 fov_shape=fov_shape, 
#                                                 frame_period=frame_period,
#                                                 plot_debug=plot_debug)

In [ ]:
# stim_type_plot = 0 # only one stim type in evoked protocol, so just set to 0 for plotting

# if trial_avg_mode == 'mean':
#     plot_resp_imgs(np.mean(resp_bsln[stim_type_plot], axis=0), np.mean(resp_resp[stim_type_plot], axis=0), np.mean(resp_diff[stim_type_plot], axis=0), j=stim_type_plot, l=0)
#     plt.figure(figsize=(6, 6), dpi=300)
#     plt.imshow(np.mean(resp_diff[stim_type_plot], axis=0), vmin=-400, vmax=400, cmap='bwr')
#     plt.axis('off')

# elif trial_avg_mode == 'median':
#     plot_resp_imgs(np.median(resp_bsln[stim_type_plot], axis=0), np.median(resp_resp[stim_type_plot], axis=0), np.median(resp_diff[stim_type_plot], axis=0), j=stim_type_plot, l=0)
#     plt.figure(figsize=(6, 6), dpi=300)
#     plt.imshow(np.median(resp_diff[stim_type_plot], axis=0), vmin=-400, vmax=400, cmap='bwr')
#     plt.axis('off')



In [ ]:
# Load track2p ROIs (cells tracked across all days from 'fake suite2p')
stat = np.load(os.path.join('data_proc/jm/jm064/track2p/matched_suite2p', session, 'suite2p', 'plane0', 'stat.npy'), allow_pickle=True)

In [ ]:
# plt.figure(figsize=(6, 6), dpi=300)

# if trial_avg_mode == 'mean':
#     plt.imshow(np.mean(resp_diff[stim_type_plot], axis=0), vmin=-400, vmax=400, cmap='bwr')
# elif trial_avg_mode == 'median':
#     plt.imshow(np.median(resp_diff[stim_type_plot], axis=0), vmin=-400, vmax=400, cmap='bwr')

# plt.axis('off')
# for cell in stat:
#     ypix = cell['ypix']
#     xpix = cell['xpix']
#     contour = np.zeros(fov_shape)
#     contour[ypix, xpix] = 1
#     plt.contour(contour, colors='k', linewidths=0.1, alpha=0.1)

In [ ]:
# # get the intensity for each ROI and plot the distribution of intensities across all ROIs
# intensities = []
# for cell in stat:
#     ypix = cell['ypix']
#     xpix = cell['xpix']
#     if trial_avg_mode == 'mean':
#         intensity = np.mean(resp_diff[stim_type_plot][:, ypix, xpix])
#     elif trial_avg_mode == 'median':
#         intensity = np.median(resp_diff[stim_type_plot][:, ypix, xpix])
#     intensities.append(intensity)

# plt.figure(figsize=(8, 2), dpi=300)
# plt.hist(intensities, bins=200)
# plt.title(f'Distribution of mean intensity in ROIs (trial_avg_mode={trial_avg_mode})')
# plt.xlabel('Mean intensity in ROI')

# Suite2p responses
Load

In [ ]:
from photostim_deve.response.io import Suite2pLoader

In [ ]:
iscell_thr = 0 # s2p cell probability (set to 'None' to filter by 'manual curation')
resp_s2p_zscore = True

In [ ]:
session_path = os.path.join('data_proc/jm/jm064/track2p/matched_suite2p/', session)

In [ ]:
s2p_loader = Suite2pLoader(ds_path=session_path, fs=fs, act_type=act_type, n_planes=n_planes)
n_rois = s2p_loader.get_n_rois()

cell_bool, cell_prob = s2p_loader.get_iscell_redcell(mode='iscell', c_idxs=np.arange(n_rois))
c_idxs = cell_bool if iscell_thr is None else np.where(cell_prob >= iscell_thr)[0]

act = s2p_loader.get_act_session(c_idxs=c_idxs)
s2p_idxs = s2p_loader.get_s2p_idxs(c_idxs=c_idxs)

In [ ]:
# zscore rows
def zscore_rows(arr):
    """Z-score each row of the input array."""
    mean = np.mean(arr, axis=1, keepdims=True)
    std = np.std(arr, axis=1, keepdims=True)
    return (arr - mean) / std

In [ ]:
def avg_5_bin(arr):
    """Average every 5 columns of the input array."""
    n_bins = arr.shape[1] // 5
    return np.array([arr[:, i*5:(i+1)*5].mean(axis=1) for i in range(n_bins)]).T

In [ ]:
def get_resp_s2p(act, stim_frames, stim_type, bsln_dur=500, resp_dur=2000, frame_period=0.033602476, plot_debug=False, resp_s2p_zscore=True):
    """ 
    Extract response traces for each stimulation trial by taking the activity traces in the pre-stim 'baseline', the 'response' and post-stim 'baseline' windows. 
    Done for each trial type.
    
    
    Parameters: 
    ---------- 
    act : np.ndarray
        Array of shape (n_neurons, n_frames) containing the activity traces for each neuron.
    stim_frames : list 
        List of frame indices for each stimulation. 
    stim_type : list 
        Evoked stim type index corresponding to each stimulation. 
    bsln_dur : int 
        Baseline duration in ms. Default is 500 ms. Applies to pre and post-stim baseline windows.
    resp_dur : int 
        Response duration in ms. Default is 2000 ms. 
    frame_period : float 
        Exact frame period from metadata used to convert from time to frame index. Default is 0.033602476 (for '30Hz' acquisition). 
    plot_debug : bool
        Whether to generate plots for debugging and sanity checking the synchronisation. Default is False.
    resp_s2p_zscore : bool
        Whether to z-score the activity traces before extracting the responses. Default is True.
        

    Returns: 
    ------- 
    resp : np.ndarray
        Array of shape (n_stim_types, n_stim_repetitions, n_neurons, resp_dur + 2*bsln_dur) containing the time series of the response (resp_dur +/- bsln_dur) of each neuron to each stimulation type on each trial. 
    """

    
    n_stim_types = len(np.unique(stim_type))
    n_stim_repetitions = len(stim_type) // n_stim_types # assuming equal number of repetitions for each stim type
    
    bsln_n_frames = int(np.ceil((bsln_dur/1000) / (frame_period))) # convert baseline duration from ms to number of frames
    resp_n_frames = int(np.ceil((resp_dur/1000) / (frame_period))) # convert response duration from ms to number of frames

    act = zscore_rows(act) if resp_s2p_zscore else act

    rand_nrn_idxs_debug = None
    if plot_debug:
        plt.figure(figsize=(10, 1), dpi=300)
        plt.imshow(zscore_rows(act), aspect='auto', cmap='Greys', vmin=0, vmax=1)
        for frame in stim_frames:
            plt.axvline(frame, color='C0', linestyle='--', linewidth=1)
        plt.title('Activity traces with stim frames for debugging synchronisation')
        plt.xlabel('Frame')
        plt.ylabel('Neuron')

        n_rand_nrn = 10
        seed = 42
        np.random.seed(seed)
        rand_nrn_idxs_debug = np.random.choice(act.shape[0], size=n_rand_nrn, replace=False)
        plt.figure(figsize=(10, 6), dpi=300)
        for i, idx in enumerate(rand_nrn_idxs_debug):
            act_proc = avg_5_bin(zscore_rows(act))
            plt.plot(act_proc[idx]-i*10, label=f'Neuron {idx}')
        for frame in stim_frames:
            plt.axvline(frame/5, color='grey', linestyle='--', linewidth=1)
        plt.title('Random subset of activity traces with stim frames (6 Hz avg downsampled)')
        plt.xlabel('Frame')
    
    # now get responses
    resp = np.zeros((n_stim_types, n_stim_repetitions, act.shape[0], bsln_n_frames*2 + resp_n_frames))

    for j in range(n_stim_types):
        stim_type_j_frames = stim_frames[stim_type == j] # get the stim frames for the current stim type 

        for k, stim_frame in enumerate(stim_type_j_frames):
            peristim_wind = (stim_frame - bsln_n_frames, 
                             stim_frame + resp_n_frames + bsln_n_frames)
            
            resp[j, k] = act[:, peristim_wind[0]:peristim_wind[1]]

    return resp, rand_nrn_idxs_debug

In [ ]:
resp, rand_nrn_idxs_debug = get_resp_s2p(act, stim_frames, stim_type, bsln_dur=bsln_dur, resp_dur=resp_dur, frame_period=frame_period, plot_debug=True)

In [ ]:
bsln_n_frames = int(np.ceil((bsln_dur/1000) / (frame_period))) # convert baseline duration from ms to number of frames

In [ ]:
# TODO: Think if to subtract the baseline trial-by-trial (as in the get_resp_imgs function) or to just z-score the activity across the whole session 

In [ ]:
# now plot the response traces for a random subset of neurons to check the extracted responses
stim_type_plot = 0 # only one stim type in evoked protocol, so just set

fig, axs = plt.subplots(1, 10, figsize=(10, 3), dpi=300, sharex=True, sharey=True)
for i in range(10):
    # plot single trials in grey
    resp_nrn = resp[stim_type_plot, :, rand_nrn_idxs_debug[i], :]
    # subtract baseline
    resp_nrn = resp_nrn - np.mean(resp_nrn[:, :bsln_n_frames], axis=1, keepdims=True)
    axs[i].plot(resp_nrn.T, color='grey', alpha=0.01)
    # plot mean across trials in color
    axs[i].plot(np.median(resp_nrn, axis=0), color=f'C{i}', label=f'Neuron {rand_nrn_idxs_debug[i]}')
    axs[i].axis('off')


In [ ]:
# now plot the response traces for a random subset of neurons to check the extracted responses (single trial, using bwr)
stim_type_plot = 0 # only one stim type in evoked protocol, so just set

fig, axs = plt.subplots(1, 10, figsize=(10, 1), dpi=600, sharex=True, sharey=True)
for i in range(10):
    # plot single trials in grey
    resp_nrn = resp[stim_type_plot, :, rand_nrn_idxs_debug[i], :]
    # subtract baseline
    resp_nrn = resp_nrn #- np.mean(resp_nrn[:, :bsln_n_frames], axis=1, keepdims=True)
    axs[i].imshow(resp_nrn, aspect='auto', cmap='bwr', vmin=-5, vmax=5)
    axs[i].axis('off')

In [ ]:
print(resp.shape)
resp_mean = np.mean(resp, axis=(0,1)) # average across trials for each stim type
resp_med = np.median(resp, axis=(0,1)) # average across trials for each stim type
resp_std = np.std(resp, axis=(0,1)) # average across trials for each stim type

In [ ]:
exp_dict = {
    's2p_idxs': s2p_idxs,
    'resp': resp,
    'resp_mean': resp_mean,
    'resp_med': resp_med,
    'resp_std': resp_std,
}

In [ ]:
output_path

In [ ]:
np.save(os.path.join(output_path, 'resp_evoked.npy'), exp_dict)